In [ ]:
import PyPDF2
import re
from bs4 import BeautifulSoup
from bs4.element import Comment
import urllib2
from urllib2 import URLError
from urllib2 import HTTPError
import socket

regex = re.compile(
        r'^(?:http|ftp)s?://' # http:// or https://
        r'(?:(?:[A-Z0-9](?:[A-Z0-9-]{0,61}[A-Z0-9])?\.)+(?:[A-Z]{2,6}\.?|[A-Z0-9-]{2,}\.?)|' #domain...
        r'localhost|' #localhost...
        r'\d{1,3}\.\d{1,3}\.\d{1,3}\.\d{1,3})' # ...or ip
        r'(?::\d+)?' # optional port
        r'(?:/?|[/?]\S+)$', re.IGNORECASE)

def isValidUrl(url):
    if regex.match(url) is not None:
        return True;
    return False

def tag_visible(element):
    if element.parent.name in ['style', 'script', 'meta', '[document]']:
        return False
    if isinstance(element, Comment):
        return False
    return True

i = 1
j = 1

def crawlLinks(SeedUrl):
    global i
    page = SeedUrl
    print 'Crawling:'+page
    try:
        pagesource = urllib2.urlopen(page, timeout = 5)
    except HTTPError, URLError:
        print("Something bad happened")
    except socket.timeout, e:
        print("Something bad happened")
    else:
        s = pagesource.read()
        soup = BeautifulSoup(s)
        divs = soup.findAll("div", {"class" : "content-inner grid_9 push_3"})
        file1 = open("text"+str(i)+".txt","wb")
        for d in divs:
            data1 = d.findAll(text = True)
            text1 = filter(tag_visible, data1)
            file1.write(" ".join(t.encode('ascii','ignore').decode('ascii').strip() for t in text1))
        i += 1
        file1.close()
        for d in divs:
            link = d.findAll('a', href = True)
            for li in link:
                if li['href'].endswith('.pdf'):
                    if isValidUrl(li['href']):
                        downPdf(li['href'])


def downPdf(SeedUrl):
    global j
    page = SeedUrl
    try:
        pagesource = urllib2.urlopen(page, timeout = 5)
    except HTTPError, URLError:
        print("Something bad happened")
    except socket.timeout, e:
        print("Something bad happened")
    else:
        s = pagesource.read()
        file = open("pdf"+str(j)+".pdf", "wb")
        file.write(s)
        file.close()
        file = open("pdf"+str(j)+".txt", "wb")
        pdfFileObj = open("pdf"+str(j)+".pdf", 'rb')
        pdfReader = PyPDF2.PdfFileReader(pdfFileObj)
        numofPages = pdfReader.numPages
        for n in range(0 , numofPages):
            pageObj = pdfReader.getPage(n)
            file.write("".join(t.encode('ascii', 'ignore').decode('ascii') for t in pageObj.extractText()))
        j += 1
        file.close()
        pdfFileObj.close()


def crawler(SeedUrl):
    global i
    page = SeedUrl
    print 'Crawling:'+page
    try:
        pagesource = urllib2.urlopen(page, timeout = 5)
    except HTTPError, URLError:
        print("Something bad happened")
    except socket.timeout, e:
        print("Something bad happened")
    else:
        s = pagesource.read()
        soup = BeautifulSoup(s)
        divs = soup.findAll("div", { "class" : "content-inner grid_9 push_3" })
        file1 = open("text"+str(i)+".txt","wb")
        for d in divs:
            data1 = d.findAll(text = True)
            text1 = filter(tag_visible, data1)
            file1.write(" ".join(t.encode('ascii','ignore').decode('ascii').strip() for t in text1))
        i += 1
        file1.close()
        forlinks = soup.findAll("div", {"id" : "menuwrapper"})
        for l in forlinks:
            link = l.findAll('a', href = True)
            for li in link:
                if li['href'].endswith('.pdf'):
                    downPdf(li['href'])
                else:
                    crawlLinks(li['href'])

crawler('http://du.ac.in/du/index.php?page=research')

In [23]:
import io
import nltk
import urllib2
import string

for i in range(1, 19):
    file1 = open('text'+str(i)+'.txt')
    line = file1.read()# Use this to read file content as a stream:
    sentences = nltk.sent_tokenize(line)
    appendFile = open('filteredtxt'+str(i)+'.txt','a')
    for sentence in sentences:
        for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
            if (pos == 'NN' or pos == 'JJ'):
                appendFile.write(" "+word)
    appendFile.close()
    
for i in range(1, 3):
    file1 = open('pdf'+str(i)+'.txt')
    line = file1.read()# Use this to read file content as a stream:
    sentences = nltk.sent_tokenize(line)
    appendFile = open('filteredpdf'+str(i)+'.txt','a')
    for sentence in sentences:
        for word,pos in nltk.pos_tag(nltk.word_tokenize(str(sentence))):
            if (pos == 'NN' or pos == 'JJ'):
                appendFile.write(" "+word)
    appendFile.close()

In [1]:
import glob
 
corpus = []
for file in glob.glob("*.txt"):
    with open(file, "r") as paper:
        corpus.append((file, paper.read()))

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tf = TfidfVectorizer(analyzer='word', ngram_range=(1,3), min_df = 0, stop_words = 'english')
tfidf_matrix =  tf.fit_transform([content for file, content in corpus])

In [3]:
from sklearn.metrics.pairwise import linear_kernel

def find_similar(tfidf_matrix, index, top_n = 5):
    cosine_similarities = linear_kernel(tfidf_matrix[index:index+1], tfidf_matrix).flatten()
    related_docs_indices = [i for i in cosine_similarities.argsort()[::-1] if i != index]
    return [(index, cosine_similarities[index]) for index in related_docs_indices][0:top_n]

In [ ]:
print(tfidf_matrix)

In [29]:
for index, score in find_similar(tfidf_matrix, 18):
       print score, index

0.713622701901 4
0.0865938285945 14
0.0437647613183 15
0.0410348448431 0
0.0368599016196 2


In [ ]:
print(tfidf_matrix*tfidf_matrix.T)